In [5]:
# Re-center the SEG list and place in destination folder

# You only need to change these lines: ------------

SEGMENTS = ['ZUNU_vida-airtree.hdr']
ImgFilename = r"z:\U-AllysonRegistration\Case 24\TLC-DICOM.nii.gz"
ApkFolder = r"z:\U-AllysonRegistration\Case 24\H-14519_JH110704_SPI-JH110704-V2_TLC_0.75_B35f_27153060\H-14519_JH110704_SPI-JH110704-V2_TLC_0.75_B35f_27153060"
DestinationFolder = r"z:\U-AllysonRegistration\Case 24"

# ------------ # No other change needed here on

# ! pip install SimpleITK
# ! python -m pip install --upgrade pip
# ! pip install numpy
import SimpleITK as sitk
import numpy as np
import os

ImgFilename = '/'.join(ImgFilename.split('\\'))
ApkFolder = '/'.join(ApkFolder.split('\\'))
DestinationFolder = '/'.join(DestinationFolder.split('\\'))

iim = sitk.ReadImage(ImgFilename)

for SEG in SEGMENTS:
    segfn = os.path.join(ApkFolder, SEG)
    segim = sitk.ReadImage(segfn)
    snda = sitk.GetArrayFromImage(segim)
    snds = np.flip(snda, axis=0)
    sndb = np.flip(snds, axis=1)
    fseg = sitk.GetImageFromArray(sndb)
    fseg.CopyInformation(iim)
    outfn = SEG[0:-4] + '-recentered.nii.gz'
    sitk.WriteImage(fseg, os.path.join(DestinationFolder, outfn))
    print('saved:', outfn)

print('Done.')

saved: ZUNU_vida-airtree-recentered.nii.gz
Done.


In [ ]:
# upsample/ up-sample/ resample/ re-sample:

# BX [mm]: RAI [108,71,217] LAI [114,220,76] 
# Origin [mm] = LPS Min: -108,-220,-217 
# ROIExtent [mm]: [222,149,141] 
# DesiredSpacing: [0.38, 0.38, 0.38]
# Desired Size [mnq] = ROIExtent ./ DesiredSpacing = round = [584,392,371]
# Number of image voxels X : Pi{CurrSpc ./ DesSpc} = X 21.45 !!!

import numpy as np

# P1 = [92.9, 185.7, 1250.6] # LAS
# P2 = [89.7, 55.4, 1121.7] # RAS
P1 = [92.9, -185.7, 1250.6] # LPS
P2 = [-89.7, -55.4, 1121.7] # LPS
Origin = np.minimum(P1,P2)
ROIExtent = np.absolute(np.subtract(P1,P2))
DesiredSpacing = 0.38
DesiredSize  = ROIExtent / DesiredSpacing #= round
DesiredSizeR = np.around(DesiredSize)

print('Size:', DesiredSizeR)
print('Origin:', Origin)
print('Spacing:', DesiredSpacing)
# print(ROIExtent)
# print(DesiredSize)

In [9]:
Y:\E-Kelly-VidaSegmentals&Sublobes\3- 20Spiromics-10ForKelly-ForERS23\IA210257_H-17900_20190805_125316_20230524_121115_export\ZUNU_vida-sublobes.hdr
"../E-Kelly-VidaSegmentals&Sublobes/3- Spiromics-20/IA210257_H-17900_20190805_125316_20230524_121115_export/"+L[0]

'../E-Kelly-VidaSegmentals&Sublobes/3- Spiromics-20/IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-sublobes.hdr'

In [35]:
import os

caseid = "MESAL-6022812-E3" # "MESAL-7013108-E3"
files = os.listdir("../D-Images/" + caseid)
for file in files:
    C1 = file.find("FRC") > -1
    C2 = file.find("80kV") > -1
    C3 = file.find("0.75mm") > -1
    C4 = file.endswith(".nii.gz")
    # print(C1, C2, C3, C4)
    # print(C1 and C2 and C3 and C4)
    if C1 and C2 and C3 and C4:
        stfile = file
    else:
        print("error: image file not found")
print(stfile)
imfn = "../D-Images/" + caseid + "/" + stfile
# print(imfn)

# import SimpleITK as sitk

# iim = sitk.ReadImage(imfn)


MESAL-6022812-E3_FRC_80kV_0.75mm_Q30f.nii.gz
../D-Images/MESAL-6022812-E3/MESAL-6022812-E3_FRC_80kV_0.75mm_Q30f.nii.gz


In [47]:
# Iterating through the MESA files in the images directory, and printing the ones without 0.75mm:

import os
import SimpleITK as sitk

caseids = os.listdir("../D-Images")
for caseid in caseids:
    # print(caseid)
    # caseid = "MESAL-6022812-E3" # "MESAL-7013108-E3"
    files = os.listdir("../D-Images/" + caseid)
    stfile = 0
    for file in files:
        C1 = file.find("FRC") > -1
        C2 = file.find("80kV") > -1
        C3 = file.find("0.75mm") > -1
        C4 = file.endswith(".nii.gz")
        # print(C1, C2, C3, C4)
        # print(C1 and C2 and C3 and C4)
        if C1 and C2 and C3 and C4:
            stfile = file
    if stfile == 0: print("error: image file not found for case: " + caseid)
    # else:
        # imfn = "../D-Images/" + caseid + "/" + stfile
        # print(stfile)
    
    # print(imfn)
    # iim = sitk.ReadImage(imfn)

error: image file not found for case: MESAL-7013841-E3
error: image file not found for case: MESAL-7015330-E3
error: image file not found for case: MESAL-7014333-E3
error: image file not found for case: MESAL-7017693-E3
error: image file not found for case: MESAL-7013191-E3
error: image file not found for case: MESAL-7019068-E3
error: image file not found for case: MESAL-7015348-E3
error: image file not found for case: MESAL-7044810-E3
error: image file not found for case: MESAL-4021100-E3


In [50]:
# Implementing the pipeline:

import os
import SimpleITK as sitk

caseids = os.listdir("../D-Images")
for caseid in ["MESAL-6022812-E3"]: #caseids:
    # print(caseid) # caseid = "MESAL-6022812-E3" # "MESAL-7013108-E3"
    files = os.listdir("../D-Images/" + caseid)
    stfile = 0
    for file in files:
        C1 = file.find("FRC") > -1
        C2 = file.find("80kV") > -1
        C3 = file.find("0.75mm") > -1
        C4 = file.endswith(".nii.gz")
        # print(C1, C2, C3, C4)
        # print(C1 and C2 and C3 and C4)
        if C1 and C2 and C3 and C4:
            stfile = file
    if stfile == 0: print("error: image file not found for case: " + caseid)
    else:
        imfn = "../D-Images/" + caseid + "/" + stfile
        # print(stfile)
        print(imfn)
        # iim = sitk.ReadImage(imfn)

        # Finding the lung masks:

        owd = os.getcwd()
        twd = "//itf-rs-store28.hpc.uiowa.edu/lss_hoffmane/Data/MESAL-3"
        os.chdir(twd)
        
        caseids = os.listdir("../D-Images")
        for caseid in ["MESAL-6022812-E3"]: #caseids:
            # print(caseid) # caseid = "MESAL-6022812-E3" # "MESAL-7013108-E3"
            files = os.listdir("../D-Images/" + caseid)
            stfile = 0
            for file in files:
                C1 = file.find("FRC") > -1
                C2 = file.find("80kV") > -1
                C3 = file.find("0.75mm") > -1
                C4 = file.endswith(".nii.gz")
                # print(C1, C2, C3, C4)
                # print(C1 and C2 and C3 and C4)
                if C1 and C2 and C3 and C4:
                    stfile = file
            if stfile == 0: print("error: image file not found for case: " + caseid)
            else:
                imfn =
        

../D-Images/MESAL-6022812-E3/MESAL-6022812-E3_FRC_80kV_0.75mm_Q30f.nii.gz


In [52]:


caseids = os.listdir()
for caseid in ["MESAL-6022812-E3"]: #caseids:
    # print(caseid) # caseid = "MESAL-6022812-E3" # "MESAL-7013108-E3"
    files = os.listdir("../D-Images/" + caseid)
    stfile = 0
    for file in files:
        C1 = file.find("FRC") > -1
        C2 = file.find("80kV") > -1
        C3 = file.find("0.75mm") > -1
        C4 = file.endswith(".nii.gz")
        # print(C1, C2, C3, C4)
        # print(C1 and C2 and C3 and C4)
        if C1 and C2 and C3 and C4:
            stfile = file
    if stfile == 0: print("error: image file not found for case: " + caseid)
    else:
        imfn = "../D-Images/" + caseid + "/" + stfile
        # print(stfile)
        print(imfn)
        # iim = sitk.ReadImage(imfn)

os.chdir(owd)
        

['MESAL-8011303-E3',
 'MESAL-0-VidaVessels',
 'MESAL-7044020-E3',
 'MESAL-3017850-E3',
 'MESAL-6010377-E3',
 'MESAL-7011792-E3',
 'MESAL-6022812-E3',
 'MESAL-7015224-E3',
 'MESAL-5018846-E3',
 'MESAL-5021944-E3',
 'MESAL-4013344-E3',
 'MESAL-4019920-E3',
 'MESAL-7015038-E3',
 'MESAL-4014944-E3',
 'MESAL-4016297-E3',
 'MESAL-7043309-E3',
 'MESAL-4013913-E3',
 'MESAL-8021880-E3',
 'MESAL-7041039-E3',
 'MESAL-7018975-E3',
 'MESAL-5012805-E3',
 'MESAL-3012735-E3',
 'MESAL-6022081-E3',
 'MESAL-7014848-E3',
 'MESAL-3015424-E3',
 'MESAL-6022634-E3',
 'MESAL-4011724-E3',
 'MESAL-5010632-E3',
 'MESAL-5011345-E3',
 'MESAL-5014140-E3',
 'MESAL-3012344-E3',
 'MESAL-8016062-E3',
 'MESAL-8014817-E3',
 'MESAL-8016100-E3',
 'MESAL-4022777-E3',
 'MESAL-6014500-E3',
 'MESAL-8021902-E3',
 'MESAL-8018472-E3',
 'MESAL-7010613-E3',
 'MESAL-7018800-E3',
 'MESAL-7013922-E3',
 'MESAL-6022103-E3',
 'MESAL-6010202-E3',
 'MESAL-7017103-E3',
 'MESAL-7015976-E3',
 'MESAL-4016530-E3',
 'MESAL-7910169-E3',
 'MESAL-30